In [1]:
import os

import pandas as pd

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

def applyParallel(dfGrouped, func):
#     series = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(dfGrouped))
    series = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return series

In [2]:
def parse_order(x):
    series = pd.Series()

    series['products'] = '_'.join(x['product_id'].values.astype(str).tolist())
    series['reorders'] = '_'.join(x['reordered'].values.astype(str).tolist())
    series['aisles'] = '_'.join(x['aisle_id'].values.astype(str).tolist())
    series['departments'] = '_'.join(x['department_id'].values.astype(str).tolist())

    series['order_number'] = x['order_number'].iloc[0]
    series['order_dow'] = x['order_dow'].iloc[0]
    series['order_hour'] = x['order_hour_of_day'].iloc[0]
    series['days_since_prior_order'] = x['days_since_prior_order'].iloc[0]

    return series

In [79]:
def parse_user(x):
    parsed_orders = x.groupby('order_id', sort=False).apply(parse_order)

    series = pd.Series()

    series['order_ids'] = ' '.join(parsed_orders.index.map(str).tolist())
    series['order_numbers'] = ' '.join(parsed_orders['order_number'].map(str).tolist())
    series['order_dows'] = ' '.join(parsed_orders['order_dow'].map(str).tolist())
    series['order_hours'] = ' '.join(parsed_orders['order_hour'].map(str).tolist())
    series['days_since_prior_orders'] = ' '.join(parsed_orders['days_since_prior_order'].map(str).tolist())

    series['product_ids'] = ' '.join(parsed_orders['products'].values.astype(str).tolist())
    series['aisle_ids'] = ' '.join(parsed_orders['aisles'].values.astype(str).tolist())
    series['department_ids'] = ' '.join(parsed_orders['departments'].values.astype(str).tolist())
    series['reorders'] = ' '.join(parsed_orders['reorders'].values.astype(str).tolist())

    series['eval_set'] = x['eval_set'].values[-1]

    return series

In [4]:
orders = pd.read_csv('../data/raw/orders_small.csv')
prior_products = pd.read_csv('../data/raw/order_products__prior_small.csv')
train_products = pd.read_csv('../data/raw/order_products__train_small.csv')
order_products = pd.concat([prior_products, train_products], axis=0)
products = pd.read_csv('../data/raw/products.csv')

In [5]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2497897,52,prior,1,1,9,NaN
1,1318871,52,prior,2,1,10,7.0
2,1261384,52,prior,3,1,10,7.0
3,2100631,52,prior,4,1,11,7.0
4,580568,52,prior,5,2,10,8.0


In [6]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,45,16349,1,1
1,145,31651,1,0
2,145,35898,2,0
3,145,26348,3,0
4,222,24852,1,1


In [7]:
df = orders.merge(order_products, how='left', on='order_id')

In [8]:
df = df.merge(products, how='left', on='product_id')

In [9]:
df['days_since_prior_order'] = df['days_since_prior_order'].fillna(0).astype(int)

In [10]:
df.isnull().sum()

order_id                    0
user_id                     0
eval_set                    0
order_number                0
order_dow                   0
order_hour_of_day           0
days_since_prior_order      0
product_id                750
add_to_cart_order         750
reordered                 750
product_name              750
aisle_id                  750
department_id             750
dtype: int64

In [11]:
null_cols = ['product_id', 'aisle_id', 'department_id', 'add_to_cart_order', 'reordered']

In [12]:
df[null_cols] = df[null_cols].fillna(0).astype(int)

In [13]:
if not os.path.isdir('../data/processed'):
        os.makedirs('../data/processed')

In [14]:
from tqdm import tqdm

In [15]:
tqdm.pandas(desc="my bar!")

In [67]:
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2497897,52,prior,1,1,9,0,12341,1,0,Hass Avocados,32,4
1,2497897,52,prior,1,1,9,0,14444,2,0,Smokehouse Almonds,117,19
2,2497897,52,prior,1,1,9,0,10441,3,0,Dry Roasted Almonds,117,19
3,2497897,52,prior,1,1,9,0,196,4,0,Soda,77,7
4,2497897,52,prior,1,1,9,0,46149,5,0,Zero Calorie Cola,77,7


In [78]:
for name, group in tqdm(df.groupby('user_id', sort=False)):
        print(name)
        print(group.head())
        print(type(group))


  0%|          | 8/2062 [00:00<01:10, 29.28it/s]

52
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2497897       52    prior             1          1                  9   
1   2497897       52    prior             1          1                  9   
2   2497897       52    prior             1          1                  9   
3   2497897       52    prior             1          1                  9   
4   2497897       52    prior             1          1                  9   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                       0       12341                  1          0   
1                       0       14444                  2          0   
2                       0       10441                  3          0   
3                       0         196                  4          0   
4                       0       46149                  5          0   

          product_name  aisle_id  department_id  
0        Hass Avocados        32              4  
1   Smo


  1%|          | 14/2062 [00:00<01:16, 26.80it/s]

     order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
525   2857517      542    prior             1          1                 16   
526   2857517      542    prior             1          1                 16   
527   2857517      542    prior             1          1                 16   
528   2857517      542    prior             1          1                 16   
529   1611651      542    prior             2          3                 12   

     days_since_prior_order  product_id  add_to_cart_order  reordered  \
525                       0       35886                  1          0   
526                       0       43352                  2          0   
527                       0       21288                  3          0   
528                       0       15361                  4          0   
529                       9       43352                  1          1   

                      product_name  aisle_id  department_id  
525  Organic Blueberries


  1%|          | 24/2062 [00:00<01:01, 33.22it/s]

...


  4%|▍         | 84/2062 [00:01<00:28, 68.67it/s]

<class 'pandas.core.frame.DataFrame'>
6879
       order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
11532   1747250     6879    prior             1          2                  2   
11533   1747250     6879    prior             1          2                  2   
11534   1747250     6879    prior             1          2                  2   
11535   1747250     6879    prior             1          2                  2   
11536    140880     6879    prior             2          1                 11   

       days_since_prior_order  product_id  add_to_cart_order  reordered  \
11532                       0       35935                  1          0   
11533                       0       22935                  2          0   
11534                       0       21288                  3          0   
11535                       0       24830                  4          0   
11536                      30       47740                  1          0   

                   


  5%|▍         | 101/2062 [00:01<00:30, 64.79it/s]


       order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
13720    679031     7815    prior             1          1                 18   
13721    679031     7815    prior             1          1                 18   
13722    679031     7815    prior             1          1                 18   
13723    679031     7815    prior             1          1                 18   
13724    679031     7815    prior             1          1                 18   

       days_since_prior_order  product_id  add_to_cart_order  reordered  \
13720                       0        2120                  1          0   
13721                       0       10613                  2          0   
13722                       0       15848                  3          0   
13723                       0       42620                  4          0   
13724                       0        3619                  5          0   

                                            product_name  ais

KeyboardInterrupt: 

In [80]:
user_data = df.groupby('user_id', sort=False).apply(parse_user).reset_index()

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2497897       52    prior             1          1                  9   
1   2497897       52    prior             1          1                  9   
2   2497897       52    prior             1          1                  9   
3   2497897       52    prior             1          1                  9   
4   2497897       52    prior             1          1                  9   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                       0       12341                  1          0   
1                       0       14444                  2          0   
2                       0       10441                  3          0   
3                       0         196                  4          0   
4                       0       46149                  5          0   

          product_name  aisle_id  department_id  
0        Hass Avocados        32              4  
1   Smokeh

KeyboardInterrupt: 

In [34]:
from joblib import Parallel, delayed
import multiprocessing

In [49]:
# def applyParallel(dfGrouped, func):
#     result = Parallel(n_jobs=-1)(delayed(func)(group) for name, group in tqdm(dfGrouped))
#     return pd.concat(result)

In [55]:
# user_data2 = applyParallel(df.groupby('user_id', sort=False), parse_user).reset_index()

JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/opt/conda/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/opt/conda/lib/python3.5/runpy.py in _run_c

In [68]:
def applyParallel(df, func):
    result = Parallel(n_jobs=-1)(delayed(func)(group) for name, group in tqdm(df.groupby('user_id', sort=False)))
    return result

In [69]:
user_data2 = applyParallel(df, parse_user)

100%|██████████| 2062/2062 [02:56<00:00, 10.08it/s]


In [71]:
user_data2[0]

order_ids                  2497897 1318871 1261384 2100631 580568 1680554...
order_numbers              1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
order_dows                 1 1 1 1 2 1 2 1 1 2 2 1 1 1 2 3 2 2 3 1 5 1 1 ...
order_hours                9 10 10 11 10 13 7 10 11 10 9 16 10 13 11 11 1...
days_since_prior_orders    0 7 7 7 8 6 8 6 7 8 7 6 7 7 8 8 6 14 8 5 30 10...
product_ids                12341_14444_10441_196_46149_35561 12341_6184_1...
aisle_ids                  32_117_117_77_77_107 32_32_117_77_77_117 77_11...
department_ids             4_19_19_7_7_19 4_4_19_7_7_19 7_19_7_4_9 4_19_1...
reorders                   0_0_0_0_0_0 1_0_1_1_1_0 1_1_1_0_0 1_1_1_1_0_1_...
eval_set                                                               train
dtype: object

In [63]:
user_data.shape

(2062, 11)

In [59]:
user_data.head()

,user_id,order_ids,order_numbers,order_dows,order_hours,days_since_prior_orders,product_ids,aisle_ids,department_ids,reorders,eval_set
0,52,2497897 1318871 1261384 2100631 580568 1680554...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,1 1 1 1 2 1 2 1 1 2 2 1 1 1 2 3 2 2 3 1 5 1 1 ...,9 10 10 11 10 13 7 10 11 10 9 16 10 13 11 11 1...,0 7 7 7 8 6 8 6 7 8 7 6 7 7 8 8 6 14 8 5 30 10...,12341_14444_10441_196_46149_35561 12341_6184_1...,32_117_117_77_77_107 32_32_117_77_77_117 77_11...,4_19_19_7_7_19 4_4_19_7_7_19 7_19_7_4_9 4_19_1...,0_0_0_0_0_0 1_0_1_1_1_0 1_1_1_0_0 1_1_1_1_0_1_...,train
1,108,2313085 2393725 3074448 2137209 1503688 278828...,1 2 3 4 5 6 7 8,0 0 1 3 0 0 0 6,13 13 18 11 17 15 10 15,0 0 15 30 30 28 30 6,4658_21137_41960_30489_38289_44359_21249_17606...,115_24_83_67_98_83_66_88_67 115_67_88_83_24_98...,7_4_4_20_7_4_6_13_20 7_20_13_4_4_7_6_20_4 12_1...,0_0_0_0_0_0_0_0_0 1_1_1_1_1_1_1_1_1 0_0_0_1_0_...,train
2,116,3370391 1115121 1221109 1765618 3307561 222423...,1 2 3 4 5 6 7,0 0 4 6 5 1 5,5 17 16 6 17 4 12,0 14 25 9 13 10 4,9605_22667_46049_25843_38906_23233 24852_13627...,121_43_107_107_37_112 24_94_94_94_54 24_43_134...,14_3_19_19_1_3 4_7_7_7_17 4_3_5 17_17_7_17_5 6...,0_0_0_0_0_0 0_0_0_0_0 1_1_0 1_0_0_0_0 0_1_0_0_...,train
3,124,1318740 490161 2638175 1660930 1923886 3051829...,1 2 3 4 5 6 7 8 9,0 2 1 1 1 1 1 1 1,22 10 10 10 8 9 8 8 8,0 30 6 7 7 21 21 14 21,16797_24184_42625_9387_39428_19478_19068_28985...,24_83_16_24_24_9_35_83_83 83_83_16_83_35_24_83...,4_4_4_4_4_9_12_4_4 4_4_4_4_12_4_4_4_4_4_4_4_4_...,0_0_0_0_0_0_0_0_0 1_1_1_1_1_1_0_0_0_0_0_0_0_0_...,train
4,170,3126026 2066764 2772634 2587424,1 2 3 4,1 1 2 4,16 9 9 18,0 30 30 30,25466_30850_36866 13060_35730 13060_33452_25783 0,115_26_45 70_55 70_64_64 0,7_7_19 11_11 11_7_7 0,0_0_0 0_0 1_0_0 0,test


In [64]:
user_data2.shape

(20620, 2)

In [66]:
user_data2.head(20)

,index,0
0,order_ids,2497897 1318871 1261384 2100631 580568 1680554...
1,order_numbers,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
2,order_dows,1 1 1 1 2 1 2 1 1 2 2 1 1 1 2 3 2 2 3 1 5 1 1 ...
3,order_hours,9 10 10 11 10 13 7 10 11 10 9 16 10 13 11 11 1...
4,days_since_prior_orders,0 7 7 7 8 6 8 6 7 8 7 6 7 7 8 8 6 14 8 5 30 10...
5,product_ids,12341_14444_10441_196_46149_35561 12341_6184_1...
6,aisle_ids,32_117_117_77_77_107 32_32_117_77_77_117 77_11...
7,department_ids,4_19_19_7_7_19 4_4_19_7_7_19 7_19_7_4_9 4_19_1...
8,reorders,0_0_0_0_0_0 1_0_1_1_1_0 1_1_1_0_0 1_1_1_1_0_1_...
9,eval_set,train


In [27]:
user_data.to_csv('../data/processed/user_data.csv', index=False)